In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
import gc
import pandas as pd
import numpy as np
import torch
import datetime
from tqdm import tqdm_notebook
from torch import nn
from torchvision import transforms
from cnn_finetune import make_model

import model as M
from dataset import SegmentationDataset
from data import *
from util import *

In [19]:
torch.cuda.is_available()
device = torch.device('cuda')

# Setup dataset

In [4]:
train_df, val_df, _ = get_dfs()

In [5]:
val_df.shape

(800, 5)

In [12]:
train_dataset = SegmentationDataset(train_df, mode='train').set_transformer((128, 128))
val_dataset = SegmentationDataset(val_df, mode='train').set_transformer()

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)

# Training

In [13]:
torch.cuda.empty_cache()

model = M.UNet(1, n_classes=1)
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

In [22]:
def train(model, n_epoch, train_iter, val_iter):
    for epoch in range(n_epoch):
        model.train()
        
        gc.collect()
        torch.cuda.empty_cache()
        
        total_loss = 0
        total_size = 0
        
        for batch_idx, (data, target) in enumerate(train_iter):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            
            # Forward
            output = model(data)
            loss = criterion(output.view(-1), target.view(-1))
            
            total_loss += loss.item()
            total_size += data.size(0)
            
            # Backward
            loss.backward()
            optimizer.step()
            
            if batch_idx % 100 == 0:
                now = datetime.datetime.now()
                print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {:.6f}'.format(
                    now,
                    epoch, batch_idx * len(data), len(train_iter.dataset),
                    100. * batch_idx / len(train_iter), total_loss / total_size))
                
        gc.collect()
        torch.cuda.empty_cache()
        
        iou = evaluate(model, val_iter, device=device)
        print('[{}] Train Epoch: {}\tIoU: {:.6f}'.format(now, epoch, iou))
    return model

In [23]:
model = train(model, 10, train_loader, val_loader)

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


[2018-09-28 23:40:43.758058] Train Epoch: 0 [0/3200 (0%)]	Average loss: 0.013861
[2018-09-28 23:41:12.310166] Train Epoch: 0 [1600/3200 (50%)]	Average loss: 0.022306
[2018-09-28 23:41:12.310166] Train Epoch: 0	IoU: 0.871207
[2018-09-28 23:41:54.978029] Train Epoch: 1 [0/3200 (0%)]	Average loss: 0.035116
[2018-09-28 23:42:23.568033] Train Epoch: 1 [1600/3200 (50%)]	Average loss: 0.023653
[2018-09-28 23:42:23.568033] Train Epoch: 1	IoU: 0.877615
[2018-09-28 23:42:58.270737] Train Epoch: 2 [0/3200 (0%)]	Average loss: 0.030590
[2018-09-28 23:43:26.863992] Train Epoch: 2 [1600/3200 (50%)]	Average loss: 0.022670
[2018-09-28 23:43:26.863992] Train Epoch: 2	IoU: 0.854933
[2018-09-28 23:44:01.691125] Train Epoch: 3 [0/3200 (0%)]	Average loss: 0.023196
[2018-09-28 23:44:30.215796] Train Epoch: 3 [1600/3200 (50%)]	Average loss: 0.022222
[2018-09-28 23:44:30.215796] Train Epoch: 3	IoU: 0.868768
[2018-09-28 23:45:04.950548] Train Epoch: 4 [0/3200 (0%)]	Average loss: 0.012286
[2018-09-28 23:45:33.56

In [24]:
save_model(model, 'unet_test')

In [47]:
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        y = model(data)
        break
y.shape

D:\Users\ns\Anaconda3\envs\chainer\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


torch.Size([16, 1, 128, 128])

In [21]:
transforms.ToPILImage?